### Add Oxford Museum of Natural History copyright acknowledgement

This script tags each image in a given directory with the following copyright acknowledgement:

**Acknowledgement: © Oxford University Museum of Natural History**

### Imports

In [11]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from os import walk


### Set Parameters

In [12]:
# Set the directory which contains the original images which
# need to be tagged with this copyright acknowledgement
input_images_dir  = '../../test/fusus_ficulneus'
#input_images_dir  = '../image_data/fusus_uniplicatus'

# Set the directory into which the tagged images will be saved
# Usually this will be the same as the input directory so the 
# original images are overwritten with their tagged versions
output_images_dir = '../../test/tagged_fusus_ficulneus'
# For testing though, you may wish to use a different directory
#output_images_dir = '../copyrighted_image_data/fusus_uniplicatus'


### Create a FossilImage class

In [13]:
class FossilImage:
    def __init__(self, image_directory, image_filename):
        self.directory = image_directory
        self.filename = image_filename
        ## Set the font_size for the copyright banner
        self.font_size = 10
        ## Set the font to be used in the copyright banner
        self.banner_font = ImageFont.truetype("../../assets/OpenSans.ttf", self.font_size)
        ## Set the bottom position for the text within the banner image
        self.banner_bottom_line_offset = 16
        ## Set the spacing between text lines within the banner image
        self.line_space_offset = 12
        self.image = Image.open(image_directory + '/' + image_filename)
        self.width, self.height = self.image.size
        self.ensureMinimumSize()
        self.setBannerSizeAndText()
        self.banner_width = self.width
        self.tagged_image = None
        self.createTaggedImage()
        
    def ensureMinimumSize(self):
        '''
        Given an image that is smaller than 144 pixels wide, 
        resize it to at least 144 pixels wide while 
        maintaining its aspect ratio.  This is desirable in 
        order to use copyright text in the 10 point font 
        size which is the minimum font size which renders 
        cleanly.
        '''
        if self.width < 144:
            new_size_width = 144
            new_size_height = int(round((self.height * 144) / self.width))
            new_size = (new_size_width, new_size_height)
            self.image = self.image.resize(new_size, Image.BICUBIC)
            self.width, self.height = self.image.size

    def setBannerSizeAndText(self):
        '''
        Given an image, find its width and use that to decide how
        many lines will be needed to write out the copyright 
        acknowledgement text.  Then, return those lines of text in
        an array.
        '''
        if self.width < 200: # 3 lines
            self.text_lines = [
                ' Acknowledgement:',
                ' © Oxford University',
                ' Museum of Natural History'
            ]
            self.banner_height = 38

        elif self.width < 600:
            self.text_lines = [
                ' Acknowledgement: © Oxford University',
                ' Museum of Natural History'
            ]
            self.banner_height = 28
        else:
            self.text_lines = [
                ' Acknowledgement: © Oxford University Museum of Natural History'
            ]
            self.banner_height = 16

    def createTaggedImage(self):
        '''
        Perfom all the steps necessary to add the copyright
        acknowledgment to the image
        '''
        bottom_line_offset = 12
        # Create a new image for the banner
        banner_image = Image.new('1', (self.banner_width, self.banner_height))
        banner_drawing = ImageDraw.Draw(banner_image)
        
        # Add copyright to the banner image
        counter = len(self.text_lines) - 1
        for line in self.text_lines:
            this_line_offset = self.banner_bottom_line_offset + (counter * self.line_space_offset)
            banner_drawing.text(
                (0, self.banner_height - this_line_offset),
                line,
                font = self.banner_font,
                fill = 1
            )
            counter -=1
        
        # Create a new image "canvas" to concantenate the original image with the banner image
        tagged_image = Image.new('RGB', (self.width, self.height + self.banner_height))

        # Paste the original image onto the finished image "canvas"
        tagged_image.paste(self.image, (0, 0))

        # Paste the banner onto the finished image "canvas" underneath the original image
        tagged_image.paste(banner_image, (0, self.height))
        
        # Set the new image into the tagged_image property
        self.tagged_image = tagged_image
    
        

### Functions

In [14]:
def getListOfFiles(directory):
    '''
    Given a directory name, return a list of all the files
    within that directory or any subdirectories.
    '''
    files = []
    for (dirpath, dirnames, filenames) in walk(directory):
        files.extend(filenames)
        break
    # Remove any hidden files from the list
    for f in files:
        if f.startswith('.'):
            files.remove(f)
    # Return the list of files
    return files


### Add the copyright banner to each image file

In [15]:
# Get the list of fossil image files from the input_images_dir
# specified in the Parameters section above.
files = getListOfFiles(input_images_dir)

# For each fossil image file, add the copyright banner, then save 
# the finished image in the output_images_dir specified in the
# Parameters section above.
for file_name in files:
    
    fosImg = FossilImage(input_images_dir, file_name)
    
    # Show the finished image (optional)
    #fosImg.tagged_image.show()

    ## Save the finished image
    file_name_without_ext = file_name[:-4]
    fosImg.tagged_image.save(output_images_dir + '/' + file_name_without_ext + '.png')
